# Collect all settled locations

In [110]:
import geopandas as gpd
import pandas as pd
import geopy
import osmnx as ox
import numpy as np
import rasterio as rio
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp').sample(1000)
location_csv = gpd.GeoDataFrame()
location_csv['name'] = iq_locations['PNameEn']
location_csv['region'] = iq_locations['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = iq_locations['Latitude']
location_csv['lon'] = iq_locations['Longitude']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = 0
location_csv['pop/cap'] = 0
location_csv.head()

,name,region,country,lat,lon,location_type,conflict_date,pop/cap
11830,Ibadah,Thi-Qar,Iraq,30.900000,46.7700,town,0,0
20172,Mukhtari,Ninewa,Iraq,36.160000,42.6700,town,0,0
20362,Bire,Ninewa,Iraq,36.840000,43.4800,town,0,0
6853,Kani Jna Upper,Sulaymaniyah,Iraq,35.580000,45.2300,town,0,0
22088,Ubied Habeb,Wassit,Iraq,32.561944,46.2825,town,0,0


# Get population numbers

In [111]:
import os
from shapely.geometry import Point
listy = []
old_cols = None
for each_file in os.listdir('iom_dtm_reports/'):
    print(each_file)
    df = pd.read_csv('iom_dtm_reports/' + each_file, usecols = ['Latitude', 'Longitude', 'Families', 'Individuals'])
    points = [Point(x,y) for x, y in zip(df.Longitude, df.Latitude)]
    df = gpd.GeoDataFrame(df)
    df.crs = iq_locations.crs
    df['geometry'] = gpd.GeoSeries(points).buffer(0.01) # expand to make sure we get matches
    df_join = gpd.sjoin(iq_locations, df, 'left')


d89.csv
d88.csv
r87.csv
r86.csv
r84.csv
r90.csv
r91.csv
r85.csv
r88.csv
r89.csv
d86.csv
d87.csv
d91.csv
d85.csv
d84.csv
d90.csv


In [112]:
#pop numbers only for starting state at round 89
df = pd.read_csv('iom_dtm_reports/d89.csv', usecols = ['Latitude', 'Longitude', 'Families', 'Individuals'])
points = [Point(x,y) for x, y in zip(df.Longitude, df.Latitude)]
df = gpd.GeoDataFrame(df)
df.crs = iq_locations.crs
df['geometry'] = gpd.GeoSeries(points).buffer(0.01) # expand to make sure we get matches
df_join = gpd.sjoin(iq_locations, df, 'left')
df_join.fillna(0, inplace = True)

_Perform spatial join, then average out values across points_

In [113]:
denom = df_join.groupby('index_right').count().reset_index()
denom_dict = {}
for i in denom.itertuples():
    denom_dict[i.index_right] = i.PCode
    
adj_families = []
adj_individuals = []
for i in df_join.itertuples():
    adj_families.append(int(round(i.Families/denom_dict[i.index_right])))
    adj_individuals.append(int(round(i.Individuals/denom_dict[i.index_right])))
    
    
df_join['adj_families'] = adj_families
df_join['adj_individuals'] = adj_individuals




In [114]:
df_join.head(10)

,PCode,PNameEn,PNameAr,Longitude_left,Latitude_left,PCodeAlt1,PNameAlt1,A1Code,A1NameEn,A2Code,A2NameEn,geometry,index_right,Latitude_right,Longitude_right,Families,Individuals,adj_families,adj_individuals
11830,IQ-P11798,Ibadah,,46.770000,30.900000,904711.0,`Ibadah,IQ-G09,Thi-Qar,IQ-D056,Suq Al-Shoyokh,POINT (46.77 30.9),0.0,0.000000,0.000000,0.0,0.0,0,0
20172,IQ-P20134,Mukhtari,,42.670000,36.160000,1509230.0,Mukhtari,IQ-G15,Ninewa,IQ-D087,Mosul,POINT (42.67 36.16),0.0,0.000000,0.000000,0.0,0.0,0,0
20362,IQ-P20324,Bire,,43.480000,36.840000,1509400.0,Bire,IQ-G15,Ninewa,IQ-D088,Shikhan,POINT (43.48 36.84),0.0,0.000000,0.000000,0.0,0.0,0,0
6853,IQ-P06843,Kani Jna Upper,,45.230000,35.580000,501720.0,Kani Jna Upper,IQ-G05,Sulaymaniyah,IQ-D033,Sulaymaniya,POINT (45.23 35.58),0.0,0.000000,0.000000,0.0,0.0,0,0
22088,IQ-P22050,Ubied Habeb,,46.282500,32.561944,1610182.0,Ubied habeb,IQ-G16,Wassit,IQ-D095,Kut,POINT (46.2825 32.561944),3556.0,32.561618,46.273848,4.0,24.0,4,24
22088,IQ-P22050,Ubied Habeb,,46.282500,32.561944,1610182.0,Ubied habeb,IQ-G16,Wassit,IQ-D095,Kut,POINT (46.2825 32.561944),2913.0,32.565856,46.274682,10.0,60.0,10,60
11706,IQ-P11674,Al-Hasaiyah,,46.474722,30.900278,904980.0,Al-hasaiyah,IQ-G09,Thi-Qar,IQ-D056,Suq Al-Shoyokh,POINT (46.474722 30.900278),3005.0,30.896843,46.465439,19.0,114.0,19,114
11706,IQ-P11674,Al-Hasaiyah,,46.474722,30.900278,904980.0,Al-hasaiyah,IQ-G09,Thi-Qar,IQ-D056,Suq Al-Shoyokh,POINT (46.474722 30.900278),3174.0,30.902103,46.478124,18.0,108.0,18,108
19111,IQ-P19073,Al Harrah,,47.520000,31.380000,1408213.0,Al Harrah,IQ-G14,Missan,IQ-D082,Qal'at Saleh,POINT (47.52 31.38),0.0,0.000000,0.000000,0.0,0.0,0,0
12811,IQ-P12779,Kani Zard,,45.330000,34.500000,1005747.0,Kani Zard,IQ-G10,Diyala,IQ-D060,Khanaqin,POINT (45.33 34.5),0.0,0.000000,0.000000,0.0,0.0,0,0


In [262]:

# starting populations are current counts at state 89
location_csv = gpd.GeoDataFrame()
location_csv['name'] = df_join['PNameEn']
location_csv['region'] = df_join['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = df_join['Latitude_left']
location_csv['lon'] = df_join['Longitude_left']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = None
location_csv['HH_pop'] = df_join['adj_families']
location_csv['individual_pop'] = df_join['adj_individuals']

In [263]:
location_csv.head()

,name,region,country,lat,lon,location_type,conflict_date,HH_pop,individual_pop
11830,Ibadah,Thi-Qar,Iraq,30.900000,46.7700,town,0,0,0
20172,Mukhtari,Ninewa,Iraq,36.160000,42.6700,town,0,0,0
20362,Bire,Ninewa,Iraq,36.840000,43.4800,town,0,0,0
6853,Kani Jna Upper,Sulaymaniyah,Iraq,35.580000,45.2300,town,0,0,0
22088,Ubied Habeb,Wassit,Iraq,32.561944,46.2825,town,0,4,24


In [169]:
def assign_round(x):
    start = 20170101
    # from http://iraqdtm.iom.int/IDPsML.aspx
    upper_dates = [20170105, 20170119, 20170202, 20170216, 
                  20170302, 20170316, 20170330, 20170413, 
                  20170427, 20170515, 20170530, 20170615,
                  20170630, 20170715, 20170730, 20170815,
                  20170830, 20170915, 20170930, 20171015, 
                  20171031, 20171115, 20171129, 20171215,
                  20171231,
                  20180115,20180131, 20180215, 20180228, 
                  20180315, 20180330, 20180415, 20180430,
                  ]
    top_val = len(upper_dates)
    offset = len(upper_dates)
    
    for i in upper_dates:
        if x > i:
            offset -= 1
            start = i
        else:
            return top_val - offset

In [192]:
# now do conflict locations

from shapely.geometry import Point
conflict_loc = pd.read_csv('data/acled_unprocessed_conflict_locations.csv')
conflict_loc_geo = gpd.GeoDataFrame(conflict_loc)
points = [Point(float(j), float(i)) for i, j in zip(conflict_loc.latitude, conflict_loc.longitude)]
geometries_acled = gpd.GeoSeries(points).buffer(0.1) # buffer is somewhat arbitrary...
# align geometries - can't know what geometry acled was using...
acled_locations = gpd.GeoDataFrame()
acled_locations['geometry'] = geometries_acled
acled_locations.crs = iq_locations.crs
acled_locations.to_crs(iq_locations.crs)
acled_locations['date'] = conflict_loc['event_date']
acled_locations.to_file('data/acled_conflict_geometries.shp')


In [253]:
acled_locations = gpd.read_file('data/acled_conflict_geometries.shp')
date_list = list(acled_locations.date)
conflict_geoms = dict()
for i in acled_locations.itertuples():
    date_val = assign_round(int(i.date[:4] + i.date[5:7] + i.date[8:]))
    if date_val not in conflict_geoms:
        conflict_geoms[date_val] = []
    conflict_geoms[date_val].append(i.geometry.buffer(0.01))

    new_geoms = dict()
for i, j in conflict_geoms.items():
    tmp = gpd.GeoSeries(j).unary_union
    new_geoms[str(i)] = (i, tmp)
    
    

In [283]:
conflict_geoms = gpd.GeoDataFrame(new_geoms).T
conflict_geoms.columns = ['date', 'geometry']
conflict_geoms.head()

,date,geometry
0,0,(POLYGON ((44.59928795456205 31.80489067674328...
1,1,(POLYGON ((47.90748795456205 30.53349067674328...
10,10,(POLYGON ((48.58308795456205 29.97469067674328...
11,11,(POLYGON ((45.03548795456205 31.99339067674327...
12,12,(POLYGON ((47.90748795456205 30.53349067674328...


In [338]:

location_csv_geo = gpd.GeoDataFrame()
points = gpd.GeoSeries([Point(i.lon, i.lat) for i in location_csv.itertuples()])

In [339]:
location_csv_geo['geometry'] = points
location_csv_geo['name'] = list(location_csv['name'])
joined = gpd.sjoin(location_csv_geo, conflict_geoms, 'left')

In [340]:
date_list = joined.groupby('name').apply(lambda x: min(x.date))
date_list = date_list.reset_index()
date_list.columns = ['name', 'date']
date_list.fillna(-1, inplace = True)
date_dict = dict()
for i in date_list.itertuples():
    date_dict[i.name] = int(i.date)

dates = []
for i in location_csv.itertuples():
    date_val = date_dict[i.name]
    if date_val < 0:
        date_val = None
    else:
        date_val = int(date_val)
    dates.append(date_val)


In [341]:
location_csv['conflict_date'] = dates

In [342]:
location_csv.head()

,name,region,country,lat,lon,location_type,conflict_date,HH_pop,individual_pop,date
11830,Ibadah,Thi-Qar,Iraq,30.900000,46.7700,town,NaN,0,0,NaN
20172,Mukhtari,Ninewa,Iraq,36.160000,42.6700,town,1.0,0,0,1.0
20362,Bire,Ninewa,Iraq,36.840000,43.4800,town,NaN,0,0,NaN
6853,Kani Jna Upper,Sulaymaniyah,Iraq,35.580000,45.2300,town,NaN,0,0,NaN
22088,Ubied Habeb,Wassit,Iraq,32.561944,46.2825,town,NaN,4,24,NaN


In [118]:
location_csv_tmp = iq_locations.copy()
#location_csv_tmp.crs = acled_locations.crs
acled_locations = gpd.read_file('data/acled_conflict_geometries.shp')

#location_csv_tmp = gpd.sjoin(location_csv_tmp, acled_locations)

In [119]:
location_csv_tmp.head()

,PCode,PNameEn,PNameAr,Longitude,Latitude,PCodeAlt1,PNameAlt1,A1Code,A1NameEn,A2Code,A2NameEn,geometry,index_right,date
20172,IQ-P20134,Mukhtari,,42.67,36.16,1509230.0,Mukhtari,IQ-G15,Ninewa,IQ-D087,Mosul,POINT (42.67 36.16),6004,2017-01-21
20088,IQ-P20050,Harada,,42.80,36.24,1508619.0,Harada,IQ-G15,Ninewa,IQ-D087,Mosul,POINT (42.8 36.24),6004,2017-01-21
20172,IQ-P20134,Mukhtari,,42.67,36.16,1509230.0,Mukhtari,IQ-G15,Ninewa,IQ-D087,Mosul,POINT (42.67 36.16),1278,2017-12-14
20088,IQ-P20050,Harada,,42.80,36.24,1508619.0,Harada,IQ-G15,Ninewa,IQ-D087,Mosul,POINT (42.8 36.24),1278,2017-12-14
20172,IQ-P20134,Mukhtari,,42.67,36.16,1509230.0,Mukhtari,IQ-G15,Ninewa,IQ-D087,Mosul,POINT (42.67 36.16),6190,2017-01-09


In [120]:
loc_list = location_csv_tmp.groupby('date').apply(lambda x: list(x.PNameEn))
conf_locs = pd.DataFrame(loc_list)
conf_locs['day'] = np.arange(len(conf_locs))


In [121]:
conf_locs.to_csv('conflict_locations_test.csv')


In [122]:
conf_locs.head()

,0,day
date,,
2017-01-01,"[Al-Saha, Hay al Jaza'ar - 334, Najaf, Al-Grai...",0
2017-01-02,"[Karadat Maryam, Thawra (1), Al Shamasiyah - 3...",1
2017-01-03,"[Karadat Maryam, Thawra (1), Al Zohor - 806, H...",2
2017-01-04,"[Hay al Jihad - 877, Al Zohor - 806, Hay al A'...",3
2017-01-05,"[Karadat Maryam, Thawra (1), Al Zohor - 806, H...",4


In [123]:
conf_locs = pd.read_csv('conflict_locations_test.csv')
date_int = [int("".join([j for j in i if j != '-'])) for i in conf_locs.date]
def assign_round(x):
    start = 20170101
    # from http://iraqdtm.iom.int/IDPsML.aspx
    upper_dates = [20170105, 20170119, 20170202, 20170216, 
                  20170302, 20170316, 20170330, 20170413, 
                  20170427, 20170515, 20170530, 20170615,
                  20170630, 20170715, 20170730, 20170815,
                  20170830, 20170915, 20170930, 20171015, 
                  20171031, 20171115, 20171129, 20171215,
                  20171231,
                  20180115,20180131, 20180215, 20180228, 
                  20180315, 20180330, 20180415, 20180430,
                  ]
    top_val = len(upper_dates)
    offset = len(upper_dates)
    
    for i in upper_dates:
        if x > i:
            offset -= 1
            start = i
        else:
            return top_val - offset
        
conf_locs['round'] = [assign_round(i) for i in date_int]


In [166]:
current = 0
listy = []
for i in conf_locs.itertuples():
    if i.round == current:
        listy.append(i)
    else:
        x = pd.DataFrame(listy)
        listy = []
        x.to_csv('flee-release/conflict_locs_stage_' + str(current) + ".csv")
        current = i.round
        
        
    #x.to_csv('conflict_locs_round_' + str(iom_round) + ".csv")

In [167]:
ls

DataPreparation_Flee.ipynb   iom_dtm_reports/
conflict_locations_test.csv  iq_locs_pop.csv
data/                        pops_pred.txt
flee-release/                rio_polygonize.py
flee_env/


In [3]:
# classify as camp or town
conflicts = pd.read_csv('flee-release/conflict_locs_stage_0.csv')
location_type_list = []
for i in location_csv.itertuples():
    if i.HH_pop > 0:
        location_type_list.append('camp')
    elif i.name in conflicts.
        location_type_list.append('town')
location_csv['location_type']

# Above was taking too long, considering option of matches for locations and worldpop raster

In [229]:
'''# 2015 projections from WorldPop, adjusted to UN estimates
pops_raster = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')
'''

"# 2015 projections from WorldPop, adjusted to UN estimates\npops_raster = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')\n"

In [514]:
'''#http://www.perrygeo.com/python-affine-transforms.html
pop_pred = []
pops_dat = pops.read().squeeze()
translator = pops.affines
with open('pops_pred.txt', 'w') as f:
    for y in range(pops_dat.shape[0]):
        for x in range(pops_dat.shape[1]):
            long, lat = translator * (x,y) # remember to swap b/c this is x,y
            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))
# takes too long, try another way'''

'#http://www.perrygeo.com/python-affine-transforms.html\npop_pred = []\npops_dat = pops.read().squeeze()\ntranslator = pops.affines\nwith open(\'pops_pred.txt\', \'w\') as f:\n    for y in range(pops_dat.shape[0]):\n        for x in range(pops_dat.shape[1]):\n            long, lat = translator * (x,y) # remember to swap b/c this is x,y\n            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))\n# takes too long, try another way'

In [521]:
'''translator = pops_raster.affine
pops_dat = pops_raster.read().squeeze()
iq_locs_pop = location_csv.copy()
pops = []
import numpy as np
for i in iq_locs_pop.itertuples():
    x = i.lon
    y = i.lat
    col, row = ~ translator * (x, y)
    col = int(round(col,0))
    row = int(round(row,0))
    pops.append(int(pops_dat[row, col] * 10))
    
location_csv['pop_density'] = np.arange(0, len(location_csv))
location_csv.to_csv('data/iq_locs_pop.csv')'''

# now getting the routes between all locations
start with just doing great circle distance, then maybe move to routing values

In [44]:
dist_iq_locations = iq_locations.copy()

In [ ]:
'''dist_df = pd.DataFrame()
for i in dist_iq_locations.itertuples():
    dist_df[i.PNameEn] = dist_iq_locations.geometry.distance(i.geometry)
dist_df.index = dist_df.columns
dist_df.to_csv('distance_matrix.csv')'''

In [45]:
trips = set()
count = 0
with open("flee-release/routes.csv", "w") as f:
    #f.write(",".join(["name1", "name2", "distance\n"]))
    for i in dist_iq_locations.itertuples():
        dists = dist_iq_locations.geometry.distance(i.geometry) * 1000
        dists = [int(round(i,0)) for i in dists]
        tups = zip(dist_iq_locations.PNameEn, dists)
        for destination, time in tups:
            key = (i.PNameEn, destination)
            if key[0] == key[1]:
                continue
            if key not in trips:
                trips.add(key)
                trips.add((key[1], key[0]))
                f.write(",".join([key[0], key[1], str(time)]) + "\n")
                if key[0] != key[1]:
                    f.write(",".join([key[1], key[0], str(time)]) + "\n")

# conflict locations for location_csv

In [ ]:
'''conflict_loc = pd.read_csv('https://data.humdata.org/hxlproxy/data.csv?url=https%3A%2F%2Fapi.acleddata.com%2Facled%2Fread.csv%3Flimit%3D0%26iso%3D368&name=ACLEDHXL&tagger-match-all=on&tagger-02-header=iso&tagger-02-tag=%23country%2Bcode&tagger-03-header=event_id_cnty&tagger-03-tag=%23event%2Bcode&tagger-05-header=event_date&tagger-05-tag=%23date%2Boccurred+&tagger-08-header=event_type&tagger-08-tag=%23event%2Btype&tagger-09-header=actor1&tagger-09-tag=%23group%2Bname%2Bfirst&tagger-10-header=assoc_actor_1&tagger-10-tag=%23group%2Bname%2Bfirst%2Bassoc&tagger-12-header=actor2&tagger-12-tag=%23group%2Bname%2Bsecond&tagger-13-header=assoc_actor_2&tagger-13-tag=%23group%2Bname%2Bsecond%2Bassoc&tagger-16-header=region&tagger-16-tag=%23region%2Bname&tagger-17-header=country&tagger-17-tag=%23country%2Bname&tagger-18-header=admin1&tagger-18-tag=%23adm1%2Bname&tagger-19-header=admin2&tagger-19-tag=%23adm2%2Bname&tagger-20-header=admin3&tagger-20-tag=%23adm3%2Bname&tagger-21-header=location&tagger-21-tag=%23loc%2Bname&tagger-22-header=latitude&tagger-22-tag=%23geo%2Blat&tagger-23-header=longitude&tagger-23-tag=%23geo%2Blon&tagger-25-header=source&tagger-25-tag=%23meta%2Bsource&tagger-27-header=notes&tagger-27-tag=%23description&tagger-28-header=fatalities&tagger-28-tag=%23affected%2Bkilled&header-row=1')
conflict_loc = conflict_loc.iloc[1:]
conflict_loc.to_csv("data/acled_unprocessed_conflict_locations.csv")'''


In [525]:
from shapely.geometry import Point
conflict_loc = pd.read_csv('data/acled_unprocessed_conflict_locations.csv')
conflict_loc_geo = gpd.GeoDataFrame(conflict_loc)
points = [Point(float(j), float(i)) for i, j in zip(conflict_loc.latitude, conflict_loc.longitude)]
geometries_acled = gpd.GeoSeries(points).buffer(0.1) # buffer is somewhat arbitrary...
# align geometries - can't know what geometry acled was using...
acled_locations = gpd.GeoDataFrame()
acled_locations['geometry'] = geometries_acled
acled_locations.crs = iq_locations.crs
acled_locations.to_crs(iq_locations.crs)
acled_locations['date'] = conflict_loc['event_date']
acled_locations.to_file('data/acled_conflict_geometries.shp')

In [ ]:
acled_temp = gpd.sjoin(iq_locations, acled_locations, 'left')
acled_temp.to_file('data/acled_joined_locations.shp')

# for now randomly assign conflict

In [527]:
#acled_temp.sample(1000).to_file('data/acled_joined_locations_sample.shp')
#pop_list  = location_csv['pop_density'] * 10 # some pop
#location_csv['pop/cap'] = [round(min(i, 0)) for i in pop_list]
pops_list = [i * 10 for i in location_csv.pop_density]
print(max(location_csv.pop_density))
location_csv['pop/cap'] = pops_list
#location_csv_new['pop/cap'] = round(location_csv_new['pop/cap'])
#location_csv = location_csv.iloc[:,:-1]
import numpy as np
conflict_list = []
conflict_type = []
for i in range(len(location_csv)):
    one = np.random.randint(0,2)
    if one > 0:
        conflict_type.append('conflict')
        one = np.random.randint(1,50)
    else:
        if np.random.randint(0,2) > 0:
            conflict_type.append('camp')
        else:
            conflict_type.append('town')
    conflict_list.append(one)
location_csv['conflict_date'] = conflict_list
location_csv['location_type'] = conflict_type
location_csv_new = location_csv.iloc[:,:-1]
location_csv_new.to_csv('flee-release/testing_locations_data.csv', index = False, header=False)


499


In [ ]:
#locations_sample = gpd.read_file('data/acled_joined_locations_sample.shp')
#conflict_locations = gpd.read_file('data/acled_joined_locations.shp')
#conflict_locations.head()
#location_csv_new['pop/cap'] = [int(i) for i in location_csv_new['pop/cap']]

In [ ]:
locations_sample = acled_temp.copy()
just_date = []
for i in locations_sample.itertuples():
    try:
        if '-' in i.date:
            just_date.append(int("".join([i for i in i.date if i != '-'])))
        else:
            just_date.append(-1)
    except:
        just_date.append(-1)
            
locations_sample['justdate'] = just_date
locations_sample.sort_values('justdate', inplace = True)
date_rank = []
datessofar = set()
current = -1
for i in locations_sample.itertuples():
    if i.justdate != -1:
        if i.justdate not in datessofar:
            current += 1
            datessofar.add(i.justdate)
        date_rank.append(current)
    else:
        date_rank.append(None)
locations_sample['conflict_start'] = date_rank
    
        
    

In [ ]:
conflict_list = []
conflicts = set()
for i in locations_sample.itertuples():
    if i.justdate >= 0:
        if str(i.Longitude) + str(i.Latitude) not in conflicts:
            conflicts.add(str(i.Longitude) + str(i.Latitude))
            conflict_list.append(i)
conflict_df = gpd.GeoDataFrame(conflict_list)
conflict_df

In [ ]:
'''location_csv_tmp = location_csv.copy()
points = [Point(i.lon, i.lat) for i in location_csv_tmp.itertuples()]
location_csv_tmp['geometry'] = points
conflict_list = []
type_list = []
for i in location_csv_tmp.itertuples():
    for j in conflict_df.itertuples():
        if i.geometry == j.geometry:
            conflict_list.append([i.geometry, j.conflict_start])
            break
    '''

#location_csv_tmp_joined = gpd.sjoin(location_csv_tmp, conflict_df, 'left')
#location_csv_tmp_joined

In [ ]:
df = pd.read_csv('flee-release/simulation_results.csv').T.reset_index()
df.columns = ['location'] +  list(df.columns[1:])
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp')
res_df = iq_locations.merge(df, left_on = 'PNameEn', right_on = 'location')
for i in range(10):
    res_df.plot(i)